** What I want to do:** Having 5 subjects in validtion set resulted in lower accuracy because I'm guessing we simply didnt have enough data to train with. Increasing training size might result in better accuracy.

I'll rerun all the steps, but save weights under different names.

## Step 1: Reset train and test data

In [2]:
%cd data/

/home/ubuntu/distracted_driver/data


In [2]:
!rm -rf train/
!rm -rf test/
!rm -rf valid/
!rm -rf sample/

In [4]:
#!unzip imgs.zip

In [5]:
!ls train/
!ls test/ | wc -l

c0  c1	c2  c3	c4  c5	c6  c7	c8  c9
79726


## Step 2 - Split images into training, test, validation and sample sets

In [3]:
# get image list
import pandas as pd
df_imglist = pd.read_csv('driver_imgs_list.csv')

In [10]:
# copy 3 subjects to validation
import numpy as np
import os,sys,shutil
from numpy import random
from numpy.random import RandomState
ddrandom = RandomState(42)
validation_subjects = ddrandom.choice(df_imglist.subject.unique(),size=3)

In [11]:
# first create validation directory if does not exist
if not os.path.isdir('valid'):
    !mkdir valid

for distraction_classes in os.listdir('train'):
    # create class sub-directory if does not exist
    if not os.path.isdir(os.path.join('valid',distraction_classes)):
        temp_filename = os.path.join('valid',distraction_classes)
        !mkdir $temp_filename
    
    # perform a check for each image in training class
    img_move_counter = 0
    for images in os.listdir(os.path.join('train',distraction_classes)):
        if (images==df_imglist[df_imglist.subject.isin(validation_subjects)]['img']).any():
            shutil.move(os.path.join('train',distraction_classes,images), 
                        os.path.join('valid',distraction_classes,images))
            img_move_counter+=1
    
    # print completion status after each class
    print ('Moved {} images in class {}:'.format(str(img_move_counter),distraction_classes))

Moved 254 images in class c7:
Moved 263 images in class c8:
Moved 292 images in class c6:
Moved 290 images in class c0:
Moved 289 images in class c2:
Moved 294 images in class c4:
Moved 289 images in class c9:
Moved 300 images in class c1:
Moved 293 images in class c3:
Moved 298 images in class c5:


In [12]:
# Copy 1% of training and validation data into sample training and validation set
# first create sample directory if does not exist
if not os.path.isdir('sample'):
    !mkdir sample
    
for directories in ['train','valid']:
    for distraction_classes in os.listdir(directories):
        
        # initialise number of samples and filenames for copying
        number_of_samples = int(0.01*len(os.listdir(os.path.join(directories,distraction_classes))))
        all_filenames = os.listdir(os.path.join(directories,distraction_classes))
        img_cp_counter=0
        
        # run loop for copying
        for images in ddrandom.choice(all_filenames,number_of_samples):
            
            # create sample directory and sub-directory to put image into if does not exist
            if not os.path.isdir(os.path.join('sample',directories)):
                temp_filename = os.path.join('sample',directories)
                !mkdir $temp_filename            
            if not os.path.isdir(os.path.join('sample',directories,distraction_classes)):
                temp_filename = os.path.join('sample',directories,distraction_classes)
                !mkdir $temp_filename
            
            shutil.copy2(os.path.join(directories,distraction_classes,images),
                        os.path.join('sample',directories,distraction_classes,images))
            
            img_cp_counter+=1
        
        # print completion status after each class
        print ('Copied {} images to sample in class {}:'.format(str(img_cp_counter),directories+'/'+distraction_classes))

Copied 17 images to sample in class train/c7:
Copied 16 images to sample in class train/c8:
Copied 20 images to sample in class train/c6:
Copied 21 images to sample in class train/c0:
Copied 20 images to sample in class train/c2:
Copied 20 images to sample in class train/c4:
Copied 18 images to sample in class train/c9:
Copied 19 images to sample in class train/c1:
Copied 20 images to sample in class train/c3:
Copied 20 images to sample in class train/c5:
Copied 2 images to sample in class valid/c7:
Copied 2 images to sample in class valid/c8:
Copied 2 images to sample in class valid/c6:
Copied 2 images to sample in class valid/c0:
Copied 2 images to sample in class valid/c2:
Copied 2 images to sample in class valid/c4:
Copied 2 images to sample in class valid/c9:
Copied 3 images to sample in class valid/c1:
Copied 2 images to sample in class valid/c3:
Copied 2 images to sample in class valid/c5:


## Step 3: Train vgg model

In [5]:
#%cd ..

/home/ubuntu/distracted_driver


In [1]:
# import utilities and packages
%matplotlib inline

from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from datetime import datetime
import re

import utils; reload(utils)
from utils import plots
from utils import save_array, load_array

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
# define locations for training, validation, test and sample sets
data_dir = "data"
train_path = "data/train/"
test_path = "data/test/"
validation_path = "data/valid/"
sample_train_path = "data/sample/train/"
sample_validation_path = "data/sample/valid/"
results_path = "data/results/"

In [3]:
batch_size = 64

In [4]:
# import the vgg16 model

#import
import vgg16; reload(vgg16)
from vgg16 import Vgg16

#instantiate
vgg = Vgg16()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [5]:
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(validation_path, batch_size=batch_size*2)

Found 19562 images belonging to 10 classes.
Found 2862 images belonging to 10 classes.


In [6]:
# this is something you need to attack next. Jeremy thinks there is no reason why 
# finetuning the final layer should lead to good results
vgg.finetune(batches)

In [7]:
# start logging time to execute
starttime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

vgg.fit(batches, val_batches, nb_epoch=1)

# log time for when the process ends
endtime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print ("timedelta = ", datetime.strptime(endtime,'%Y-%m-%d %H:%M:%S') - datetime.strptime(starttime,'%Y-%m-%d %H:%M:%S'))

Epoch 1/1
19562/19562 [==============================] - 585s - loss: 1.8564 - acc: 0.6118 - val_loss: 1.7817 - val_acc: 0.5692
timedelta =  0:09:56


In [8]:
# save weights of the 1st epoch
vgg.model.save_weights(results_path+'3subjects_validation_ft_1epoch.h5')

In subsequent iterations, clip your model probabilities to be less sure of an answer. This is because your model accuracy is shite anyway. 

Also, finetune one level below the final layer and see if the result improves. 

If it does, go one level lower.

Also, compute log loss on validation set to see how strong it is. Also compute confusion matrix to visualise how bad things are.

## Step 4: Validate model

Skip this part in this iteration. Go to the submission part.

## Step 5: Predict on test set

In [5]:
vgg.model.load_weights(results_path+'3subjects_validation_ft_1epoch.h5')

In [7]:
# for the vgg.test method to run, all images need to be part of another class.
# Lets call it 'unknown'
tempdir = test_path+'unknown'
!mkdir $tempdir

In [8]:
img_mv_counter = 0
for img in os.listdir(test_path):
    if os.path.isfile(os.path.join(test_path,img)):
        os.rename(os.path.join(test_path,img),os.path.join(test_path,'unknown',img))
        img_mv_counter+=1
print ("{} images moved to unknown folder.".format(str(img_mv_counter)))

79726 images moved to unknown folder.


In [13]:
# start logging time to execute
starttime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

test_batches, test_proba = vgg.test(test_path, batch_size = 64)

# log time for when the process ends
endtime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print ("timedelta = ", datetime.strptime(endtime,'%Y-%m-%d %H:%M:%S') - datetime.strptime(starttime,'%Y-%m-%d %H:%M:%S'))

Found 79726 images belonging to 1 classes.
timedelta =  0:34:04


In [14]:
save_array(results_path + '3subjects_validation_test_preds.dat', test_proba)
save_array(results_path + '3subjects_validation_filenames.dat', test_batches.filenames)

## Step 6: Prepare final submission

In [15]:
import pandas as pd
result_cols = ['c'+str(i) for i in np.arange(10)]
df_results = pd.concat(
    [pd.DataFrame(test_batches.filenames, columns=['img']),
    pd.DataFrame(test_proba,columns=result_cols)], axis=1
)
print (df_results.shape)
df_results.head()

(79726, 11)


,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,unknown/img_1.jpg,1.048789e-05,9.500974e-05,6.204220e-04,5.290291e-04,5.379053e-05,3.208719e-01,8.911347e-07,6.649228e-05,0.147463,5.302886e-01
1,unknown/img_10.jpg,2.243764e-01,1.297860e-03,6.529220e-07,6.650519e-06,7.053635e-01,8.519089e-03,4.642937e-03,3.558198e-06,0.051927,3.862250e-03
2,unknown/img_100.jpg,9.649227e-01,3.491086e-02,3.665324e-10,9.536329e-14,3.299019e-10,4.643474e-09,8.743167e-07,1.611529e-04,0.000004,6.297035e-07
3,unknown/img_1000.jpg,8.633835e-09,1.010079e-08,8.227801e-05,8.352095e-07,1.391661e-02,5.483331e-06,2.091768e-06,4.222639e-08,0.985993,1.038294e-07
4,unknown/img_100000.jpg,4.752517e-03,6.385883e-04,3.757137e-03,5.996618e-01,5.530323e-02,3.357163e-05,1.182646e-02,1.786151e-06,0.293674,3.035127e-02


In [16]:
# removing the unknown tag from the image name
df_results['img'] = df_results.img.apply(lambda x:x.split('/')[1])

# save to results
df_results.to_csv(results_path+'3subjects_validation_1epoch_submission2.csv',index=False)

from IPython.display import FileLink
FileLink(results_path+'3subjects_validation_1epoch_submission2.csv')

/home/ubuntu/distracted_driver/data/results/3subjects_validation_1epoch_submission2.csv

This results in a score of 2.02692. Minor improvement over previous, but still need A LOT of work.